In [1]:
import sys

preprocessing_path = 'C:\\Users\\joshh\\Desktop\\Uni\\Soton Year 3\\COMP3200\\fake-news-profiling\\classifier\\preprocessing'
if preprocessing_path not in sys.path:
    sys.path.insert(1, preprocessing_path)

notif_path = 'C:\\Users\\joshh\\Desktop\\Uni\\Soton Year 3\\COMP3200\\fake-news-profiling\\classifier\\notifications'
if notif_path not in sys.path:
    sys.path.insert(1, notif_path)

In [2]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

import ipynb.fs.full.parse_datasets as datasets
import ipynb.fs.full.preprocessing as pp
import ipynb.fs.full.bert_fake_news_classifier as bclf
from ipynb.fs.full.notif_email import send_email

## Parse the dataset

In [3]:
tweet_data, label_data = datasets.parse_dataset("datasets", "en")

## Preprocess the data

In [4]:
# Preprocess the data
tweet_preprocessor = pp.TweetPreprocessor(
    preprocess_funcs = [
        pp.tag_indicators,
        pp.replace_xml_and_html,
        pp.replace_emojis,
        pp.remove_punctuation,
        pp.replace_tags,
        pp.remove_hashtag_chars,
        pp.replace_accented_chars,
        pp.tag_numbers,
        pp.remove_stopwords,
        pp.remove_extra_spacing,
    ])
tweet_preprocessor.preprocess(tweet_data)

In [5]:
tweet_data_individual = tweet_preprocessor.get_individual_tweets_dataset()
tweet_data_feed = tweet_preprocessor.get_tweet_feed_dataset()

In [6]:
# Split the data
(tweet_train, label_train, 
 tweet_val, label_val, 
 tweet_test, label_test) = datasets.split_dataset(tweet_data_individual, label_data)

## BERT Individual Model

In [7]:
small_bert_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1"
bert_encoder_individual = hub.KerasLayer(
    small_bert_url, 
    trainable=True,
)

bert_input_size_individual = 128

In [8]:
individual_tokenizer = bclf.BertIndividualTweetTokenizer(bert_encoder_individual, bert_input_size_individual)
bert_model_individual = bclf.create_bert_model(bert_encoder_individual, bert_input_size_individual)
bert_model_individual.compile(Adam(lr=1e-5), 'binary_crossentropy', ['accuracy'])

In [9]:
# Create a checkpoint for training
checkpoint_path_individual = "training/bert_training_individual_1/cp.ckpt"

bert_checkpoint_callback_individual = ModelCheckpoint(
    filepath=checkpoint_path_individual,
    save_weights_only=True,
    verbose=1
)

In [10]:
tweet_individual_train = individual_tokenizer.tokenize_input(tweet_train)
label_individual_train = individual_tokenizer.tokenize_labels(label_train)
tweet_individual_val = individual_tokenizer.tokenize_input(tweet_val)
label_individual_val = individual_tokenizer.tokenize_labels(label_val)
tweet_individual_test = individual_tokenizer.tokenize_input(tweet_test)
label_individual_test = individual_tokenizer.tokenize_labels(label_test)

### Finding optimal hyper parameters (batch_size, epochs)

In [13]:
results = [{'batch_size': 1, 'epochs': 10, 'loss': 1.2027349472045898, 'accuracy': 0.570888876914978}, {'batch_size': 8, 'epochs': 10, 'loss': 0.8909910321235657, 'accuracy': 0.5973333120346069}, {'batch_size': 8, 'epochs': 50, 'loss': 2.1527915000915527, 'accuracy': 0.5951111316680908}, {'batch_size': 8, 'epochs': 100, 'loss': 2.6716349124908447, 'accuracy': 0.5737777948379517}, {'batch_size': 32, 'epochs': 10, 'loss': 0.8014382719993591, 'accuracy': 0.5855555534362793}, {'batch_size': 32, 'epochs': 50, 'loss': 1.791918396949768, 'accuracy': 0.5933333039283752}, {'batch_size': 32, 'epochs': 100, 'loss': 2.2307729721069336, 'accuracy': 0.5973333120346069}]
batch_sizes = [64]
epochs = [10, 50, 100]

for batch_size in batch_sizes:
    for epoch in epochs:

        # Fit the model and then evaluate
        with tf.device('gpu:0'):
            bert_encoder_individual_test = hub.KerasLayer(
                small_bert_url, 
                trainable=True,
            )
            bert_model_individual_test = clf.create_bert_model(bert_encoder_individual_test, bert_input_size_individual)
            bert_model_individual_test.compile(Adam(lr=1e-5), 'binary_crossentropy', ['accuracy'])
            
            bert_model_individual_test.fit(
                x=tweet_individual_train, 
                y=label_individual_train, 
                batch_size=batch_size, 
                epochs=epoch,
            )
            
            evaluated_results = bert_model_individual_test.evaluate(tweet_individual_val, label_individual_val)
            results.append({
                'batch_size': batch_size, 
                'epochs': epoch, 
                'loss': evaluated_results[0], 
                'accuracy': evaluated_results[1]
            })
            print(results[-1])

Epoch 1/10
329/329 [==============================] - 81s 246ms/step - loss: 0.6682 - accuracy: 0.5874
Epoch 2/10
329/329 [==============================] - 81s 247ms/step - loss: 0.6340 - accuracy: 0.6392
Epoch 3/10
329/329 [==============================] - 81s 247ms/step - loss: 0.6187 - accuracy: 0.6549
Epoch 4/10
329/329 [==============================] - 81s 247ms/step - loss: 0.5999 - accuracy: 0.6768
Epoch 5/10
329/329 [==============================] - 81s 247ms/step - loss: 0.5834 - accuracy: 0.6926
Epoch 6/10
329/329 [==============================] - 81s 247ms/step - loss: 0.5652 - accuracy: 0.7097
Epoch 7/10
329/329 [==============================] - 81s 246ms/step - loss: 0.5473 - accuracy: 0.7217
Epoch 8/10
329/329 [==============================] - 81s 247ms/step - loss: 0.5304 - accuracy: 0.7325
Epoch 9/10
329/329 [==============================] - 81s 247ms/step - loss: 0.5142 - accuracy: 0.7476
Epoch 10/10
141/141 [==============================] - 6s 42ms/step - los

329/329 [==============================] - 81s 247ms/step - loss: 0.0514 - accuracy: 0.9796
Epoch 95/100
329/329 [==============================] - 81s 247ms/step - loss: 0.0529 - accuracy: 0.9806
Epoch 96/100
329/329 [==============================] - 81s 247ms/step - loss: 0.0484 - accuracy: 0.9811
Epoch 97/100
329/329 [==============================] - 81s 247ms/step - loss: 0.0541 - accuracy: 0.9789
Epoch 98/100
329/329 [==============================] - 81s 247ms/step - loss: 0.0518 - accuracy: 0.9798
Epoch 99/100
329/329 [==============================] - 81s 247ms/step - loss: 0.0523 - accuracy: 0.9791
Epoch 100/100
141/141 [==============================] - 6s 42ms/step - loss: 2.0028 - accuracy: 0.6007
{'batch_size': 64, 'epochs': 100, 'loss': 2.0028185844421387, 'accuracy': 0.6006666421890259}


In [21]:
best_result = max(results, key=lambda result: result['accuracy'])
best_result

{'batch_size': 64,
 'epochs': 10,
 'loss': 0.725799024105072,
 'accuracy': 0.6437777876853943}

In [15]:
send_email(
    f"""
    Grid Search finished.
    Best model: 
    > batch_size: {best_result['batch_size']}
    > epochs: {best_result['epochs']}
    > loss: {best_result['loss']}
    > accuracy: {best_result['accuracy']}
    
    All results: {results}
    
    Now training model.
    """)

In [ ]:
results = [{'batch_size': 1,
  'epochs': 10,
  'loss': 1.2027349472045898,
  'accuracy': 0.570888876914978},
 {'batch_size': 8,
  'epochs': 10,
  'loss': 0.8909910321235657,
  'accuracy': 0.5973333120346069},
 {'batch_size': 8,
  'epochs': 50,
  'loss': 2.1527915000915527,
  'accuracy': 0.5951111316680908},
 {'batch_size': 8,
  'epochs': 100,
  'loss': 2.6716349124908447,
  'accuracy': 0.5737777948379517},
 {'batch_size': 32,
  'epochs': 10,
  'loss': 0.8014382719993591,
  'accuracy': 0.5855555534362793},
 {'batch_size': 32,
  'epochs': 50,
  'loss': 1.791918396949768,
  'accuracy': 0.5933333039283752},
 {'batch_size': 32,
  'epochs': 100,
  'loss': 2.2307729721069336,
  'accuracy': 0.5973333120346069},
 {'batch_size': 64,
  'epochs': 10,
  'loss': 0.725799024105072,
  'accuracy': 0.6437777876853943},
 {'batch_size': 64,
  'epochs': 50,
  'loss': 1.4171788692474365,
  'accuracy': 0.6006666421890259},
 {'batch_size': 64,
  'epochs': 100,
  'loss': 2.0028185844421387,
  'accuracy': 0.6006666421890259}]

#### Fitting with optimal parameters

In [16]:
with tf.device('gpu:0'):
    # Fit
    bert_model_individual.fit(
        x=tweet_individual_train, 
        y=label_individual_train, 
        batch_size=best_result['batch_size'], 
        epochs=best_result['epochs'], 
        callbacks=[bert_checkpoint_callback_individual],
        validation_data=(tweet_individual_val, label_individual_val),
    )

Epoch 1/10
328/329 [============================>.] - ETA: 0s - loss: 0.6594 - accuracy: 0.6004
Epoch 00001: saving model to training/bert_training_individual_1\cp.ckpt
329/329 [==============================] - 88s 266ms/step - loss: 0.6594 - accuracy: 0.6003 - val_loss: 0.6572 - val_accuracy: 0.6369
Epoch 2/10
328/329 [============================>.] - ETA: 0s - loss: 0.6308 - accuracy: 0.6431
Epoch 00002: saving model to training/bert_training_individual_1\cp.ckpt
329/329 [==============================] - 87s 265ms/step - loss: 0.6309 - accuracy: 0.6430 - val_loss: 0.6558 - val_accuracy: 0.6418
Epoch 3/10
328/329 [============================>.] - ETA: 0s - loss: 0.6128 - accuracy: 0.6644
Epoch 00003: saving model to training/bert_training_individual_1\cp.ckpt
329/329 [==============================] - 87s 265ms/step - loss: 0.6129 - accuracy: 0.6643 - val_loss: 0.6577 - val_accuracy: 0.6456
Epoch 4/10
328/329 [============================>.] - ETA: 0s - loss: 0.5971 - accuracy: 0.

In [17]:
eval_result = bert_model_individual.evaluate(tweet_individual_test, label_individual_test)

141/141 [==============================] - 6s 42ms/step - loss: 0.7464 - accuracy: 0.5804


In [18]:
send_email(
    f"""
    Individual model fit finished.
    > loss: {eval_result[0]}
    > accuracy: {eval_result[1]}
    """)

### Evaluating for each user (rather than each tweet)

In [11]:
# Calculate user predictions from individual tweet data
def calculate_user_predictions_from_individual_tweets(model, tweets, labels):
    # Get tweets for each user
    tweets_per_user = 100
    user_tweets = [
        {
            'input_word_ids': tweets['input_word_ids'][i:i+tweets_per_user],
            'input_mask': tweets['input_mask'][i:i+tweets_per_user],
            'input_type_ids': tweets['input_type_ids'][i:i+tweets_per_user],
        }
        for i in range(0, len(tweets['input_word_ids']), tweets_per_user)
    ]
    user_labels = np.asarray([
        labels[i].numpy() for i in range(0, len(labels), tweets_per_user)
     ])
    
    # Evaluate each user
    all_predictions = []
    for user_label, user_tweet in zip(user_labels, user_tweets):
        all_predictions.append(
            model.predict(user_tweet).flatten()
        )
    
    return np.asarray(all_predictions), user_labels

In [12]:
def is_true_positive(label, prediction):
    return label == 1 and prediction == 1
    
def is_false_positive(label, prediction):
    return label == 0 and prediction == 1

def is_false_negative(label, prediction):
    return label == 1 and prediction == 0 
    
def is_true_negative(label, prediction):
    return label == 0 and prediction == 0

# Evaluate the model, returning accuracy, recall, f1, etc
# predictions should be of type [(label, [predictions])]
def evaluate_model(predictions, labels):
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    for prediction, label in zip(predictions, labels):
        # Take the mean of the users predictions and compare to threshold
        if is_true_positive(label, prediction):
            tp += 1
        elif is_false_positive(label, prediction):
            fp += 1
        elif is_false_negative(label, prediction):
            fn += 1
        elif is_true_negative(label, prediction):
            tn += 1
        else:
            print("Error:", label, prediction)
    
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else float("NaN")
    recall = tp / (tp + fn) if (tp + fn) > 0 else float("NaN")
    f1 = 2 * (precision * recall) / (precision + recall) if precision > 0 and recall > 0 else float("NaN")
            
    return {
        'true_positives': tp,
        'false_positives': fp,
        'false_negatives': fn,
        'true_negatives': tn,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [98]:
def create_test_model(trainable=True):
    bert_encoder_individual_test = hub.KerasLayer(
        small_bert_url, 
        trainable=trainable,
    )
    bert_model_individual_test = bclf.create_bert_model(bert_encoder_individual_test, bert_input_size_individual)
    bert_model_individual_test.compile(Adam(lr=1e-5), 'binary_crossentropy', ['accuracy'])
    return bert_encoder_individual_test, bert_model_individual_test

In [95]:
# Train and save best individual models
pairs = [(8, 10), (32, 100), (64, 10)]

for batch_size, epoch in pairs:
    with tf.device('gpu:0'):
        # Create checkpoint
        checkpoint = ModelCheckpoint(
            filepath=f"training/bert_individual/batch{batch_size}-epoch{epoch}-2/cp.ckpt",
            save_weights_only=True,
            verbose=1
        )
        
        # Create model
        bert_encoder_individual_test = hub.KerasLayer(
            small_bert_url, 
            trainable=True,
        )
        bert_model_individual_test = bclf.create_bert_model(bert_encoder_individual_test, bert_input_size_individual)
        bert_model_individual_test.compile(Adam(lr=1e-5), 'binary_crossentropy', ['accuracy'])

        bert_model_individual_test.fit(
            x=tweet_individual_train, 
            y=label_individual_train, 
            batch_size=batch_size, 
            epochs=epoch, 
            callbacks=[checkpoint],
        )

Epoch 1/10
   1/2625 [..............................] - ETA: 1:10 - loss: 0.6308 - accuracy: 0.7500WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0160s vs `on_train_batch_end` time: 0.0290s). Check your callbacks.


2625/2625 [==============================] - ETA: 0s - loss: 0.6508 - accuracy: 0.6153
Epoch 00001: saving model to training/bert_individual/batch8-epoch10-2\cp.ckpt
2625/2625 [==============================] - 107s 41ms/step - loss: 0.6508 - accuracy: 0.6153
Epoch 2/10
2625/2625 [==============================] - ETA: 0s - loss: 0.6046 - accuracy: 0.6673
Epoch 00002: saving model to training/bert_individual/batch8-epoch10-2\cp.ckpt
2625/2625 [==============================] - 108s 41ms/step - loss: 0.6046 - accuracy: 0.6673
Epoch 3/10
2625/2625 [==============================] - ETA: 0s - loss: 0.5679 - accuracy: 0.7029
Epoch 00003: saving model to training/bert_individual/batch8-epoch10-2\cp.ckpt
2625/2625 [==============================] - 107s 41ms/step - loss: 0.5679 - accuracy: 0.7029
Epoch 4/10
2625/2625 [==============================] - ETA: 0s - loss: 0.5316 - accuracy: 0.7358
Epoch 00004: saving model to training/bert_individual/batch8-epoch10-2\cp.ckpt
2625/2625 [==========

656/657 [============================>.] - ETA: 0s - loss: 0.0753 - accuracy: 0.9708
Epoch 00051: saving model to training/bert_individual/batch32-epoch100-2\cp.ckpt
657/657 [==============================] - 87s 132ms/step - loss: 0.0753 - accuracy: 0.9708
Epoch 52/100
656/657 [============================>.] - ETA: 0s - loss: 0.0790 - accuracy: 0.9698
Epoch 00052: saving model to training/bert_individual/batch32-epoch100-2\cp.ckpt
657/657 [==============================] - 87s 132ms/step - loss: 0.0790 - accuracy: 0.9698
Epoch 53/100
656/657 [============================>.] - ETA: 0s - loss: 0.0745 - accuracy: 0.9726
Epoch 00053: saving model to training/bert_individual/batch32-epoch100-2\cp.ckpt
657/657 [==============================] - 87s 132ms/step - loss: 0.0745 - accuracy: 0.9726
Epoch 54/100
656/657 [============================>.] - ETA: 0s - loss: 0.0681 - accuracy: 0.9737
Epoch 00054: saving model to training/bert_individual/batch32-epoch100-2\cp.ckpt
657/657 [============

In [154]:
send_email("Finished training")

#### Classifying BERT predictions

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [96]:
def train_classifier(clf, X_train, y_train, X_val, y_val, batch_size, epochs):
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_val)
    return {
        'batch_size': batch_size,
        'epochs': epochs,
        'predictions_val': predictions,
        'labels_val': y_val,
    }

In [99]:
pairs = [(8, 10), (32, 100), (64, 10)]
predictions = {
    'logistic_regression': [],
    'logistic_regression_sorted': [],
    'svm': [],
    'svm_sorted': [],
}

for batch_size, epoch in pairs:
    # Load the BERT model
    print(f"Loading weights for batch_size:{batch_size}, epochs: {epoch}")
    encoder, model = create_test_model(trainable=False)
    model.load_weights(
        f"training/bert_individual/batch{batch_size}-epoch{epoch}-2/cp.ckpt"
    ).expect_partial()
    
    # Predict training and validation set data
    X_train, y_train = calculate_user_predictions_from_individual_tweets(
        model, 
        tweet_individual_train, 
        label_individual_train,
    )
    X_val, y_val = calculate_user_predictions_from_individual_tweets(
        model, 
        tweet_individual_val, 
        label_individual_val,
    )
    
    # Train Logistic Regression model
    print("Training LR")
    predictions['logistic_regression'].append(
        train_classifier(
            LogisticRegression(), 
            X_train, 
            y_train, 
            X_val, 
            y_val, 
            batch_size, 
            epoch
        )
    )
    
    # Train Logistic Regression when training data sorted
    print("Training LR sorted")
    X_train_sorted = np.sort(X_train, axis=1)
    X_val_sorted = np.sort(X_val, axis=1)
    predictions['logistic_regression_sorted'].append(
        train_classifier(
            LogisticRegression(), 
            X_train_sorted, 
            y_train, 
            X_val_sorted, 
            y_val, 
            batch_size, 
            epoch
        )
    )

    # Train SVM model
    print("Training SVM")
    predictions['svm'].append(
        train_classifier(
            SVC(probability=True), 
            X_train, 
            y_train, 
            X_val, 
            y_val, 
            batch_size, 
            epoch
        )
    )

    # Train SVM when training data sorted
    print("Training SVM sorted")
    predictions['svm_sorted'].append(
        train_classifier(
            SVC(probability=True), 
            X_train_sorted, 
            y_train, 
            X_val_sorted, 
            y_val, 
            batch_size, 
            epoch
        )
    )

Loading weights for batch_size:8, epochs: 10
Training LR
Training LR sorted
Training SVM
Training SVM sorted
Loading weights for batch_size:32, epochs: 100
Training LR
Training LR sorted
Training SVM
Training SVM sorted
Loading weights for batch_size:64, epochs: 10
Training LR
Training LR sorted
Training SVM
Training SVM sorted


In [47]:
send_email(f"Finished training, predictions:\n{predictions}")
predictions

{'logistic_regression': [{'batch_size': 8,
   'epochs': 10,
   'predictions_val': array([0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
          1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
          1]),
   'labels_val': array([0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
          1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
          1])},
  {'batch_size': 32,
   'epochs': 100,
   'predictions_val': array([0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
          1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
          1]),
   'labels_val': array([0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
          1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
          1])},
  {'batch_size': 64,
   'epochs': 10,
   'predictions_val': array([0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
          1, 1, 0, 0, 1, 0, 1, 1

In [100]:
import pandas as pd

results = []
for key, val in predictions.items():
    for clf_info in val:
        res = []
        res.append(key)
        res.append(clf_info['batch_size'])
        res.append(clf_info['epochs'])
        
        eval_scores = evaluate_model(
            clf_info['predictions_val'], 
            clf_info['labels_val']
        )
        res.append(eval_scores['accuracy'])
        res.append(eval_scores['precision'])
        res.append(eval_scores['recall'])
        res.append(eval_scores['f1'])
        results.append(res)

df = pd.DataFrame(results, columns=['final classifier', 'batch_size', 'epochs', 'accuracy', 'precision', 'recall', 'f1'])

In [101]:
df.sort_values(by=['accuracy', 'f1'], ascending=False)

,final classifier,batch_size,epochs,accuracy,precision,recall,f1
1,logistic_regression,32,100,0.777778,0.750000,0.875000,0.807692
9,svm_sorted,8,10,0.755556,0.782609,0.750000,0.765957
7,svm,32,100,0.733333,0.772727,0.708333,0.739130
4,logistic_regression_sorted,32,100,0.711111,0.720000,0.750000,0.734694
2,logistic_regression,64,10,0.711111,0.739130,0.708333,0.723404
10,svm_sorted,32,100,0.711111,0.739130,0.708333,0.723404
0,logistic_regression,8,10,0.711111,0.761905,0.666667,0.711111
3,logistic_regression_sorted,8,10,0.711111,0.761905,0.666667,0.711111
8,svm,64,10,0.711111,0.789474,0.625000,0.697674
5,logistic_regression_sorted,64,10,0.711111,0.823529,0.583333,0.682927


In [ ]:
send_email("Finished")

#### Predict the probability of a user being a fake news spreader

In [31]:
# Load the BERT model
print(f"Loading weights for batch_size: 32, epochs: 100")
encoder, model = create_test_model()
model.load_weights(
    f"training/bert_individual/batch32-epoch100/cp.ckpt"
).expect_partial()

# Predict training and validation set data
X_train, y_train = calculate_user_predictions_from_individual_tweets(
    model, 
    tweet_individual_train, 
    label_individual_train,
)
X_val, y_val = calculate_user_predictions_from_individual_tweets(
    model, 
    tweet_individual_val, 
    label_individual_val,
)

# Train Logistic Regression model
print("Training LR")
log_reg_clf = LogisticRegression()
log_reg_clf.fit(X_train, y_train)

# Train SVM
print("Training SVM")
svm_clf = SVC(probability=True)
svm_clf.fit(X_train, y_train)

# Train Logistic Regression when training data sorted
print("Training LR sorted")
X_train_sorted = np.sort(X_train, axis=1)
X_val_sorted = np.sort(X_val, axis=1)
log_reg_sorted_clf = LogisticRegression()
log_reg_sorted_clf.fit(X_train_sorted, y_train)

Loading weights for batch_size: 32, epochs: 100


Training LR
Training SVM
Training LR sorted


LogisticRegression()

In [32]:
print("LR User 1 predict probability:", 
      log_reg_clf.predict_proba([X_val[0]]))
print("LR User 1 prediction:", log_reg_clf.predict([X_val[0]]))
print("LR sorted User 1 predict probability:", 
      log_reg_sorted_clf.predict_proba([X_val_sorted[0]]))
print("LR sorted User 1 prediction:", log_reg_sorted_clf.predict([X_val_sorted[0]]))
print("SVM User 1 predict probability:", 

      svm_clf.predict_proba([X_val[0]]))
print("SVM User 1 prediction:", svm_clf.predict([X_val[0]]))
print("User 1 label:", y_val[0])

LR User 1 predict probability: [[0.52118894 0.47881106]]
LR User 1 prediction: [0]
LR sorted User 1 predict probability: [[0.74938058 0.25061942]]
LR sorted User 1 prediction: [0]
SVM User 1 predict probability: [[0.792287 0.207713]]
SVM User 1 prediction: [0]
User 1 label: 0


### Training BERT individual + Logistic Regression sorted
BERT Model:
* BERT L-12, Input 128
* Individual tweets
* batch_size 32, epochs 100

Logistic Regression Model:
* Predict training set using BERT and sort each datapoint - this will be the LR training data
* Logistic Regression

In [22]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

def test_bert_model(encoder, input_size, dropout=0.1):
    # Create BERT input layers
    def input_layer(input_name):
        return Input(shape=(input_size,), dtype=tf.int32, name=input_name)

    inputs = {
        'input_word_ids': input_layer("inputs/input_word_ids"),
        'input_mask': input_layer("inputs/input_mask"),
        'input_type_ids': input_layer("inputs/input_type_ids"),
    }

    # BERT's pooled output
    encoder_pooled_output = encoder(inputs)['pooled_output']
    
    # Dropout
    dropout_out = tf.keras.layers.Dropout(dropout)(encoder_pooled_output)

    # Dense layer output
    dense_output = Dense(1, activation='sigmoid')(dropout_out)

    # Create the Keras model and compile
    return Model(inputs, dense_output)

In [15]:
from official.nlp import optimization

In [ ]:
setup = [(b, e, lr) 
         for b in [16, 32, 48, 64] 
         for e in [10] 
         for lr in [5e-5, 3e-5, 2e-5]]

for batch_size, epochs, learning_rate in setup:
    
    # TensorBoard callback for logging loss
    model_name = f"batch_size{batch_size}-epochs{epochs}-lr{learning_rate}"
    model_path = "training/bert_individual/loss-testing-4"
    log_dir = model_path + "logs/" + model_name
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    
    # Checkpoint to save model
    checkpoint_path_individual = model_path + model_name + "/cp.ckpt"
    bert_individual_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_path_individual,
        save_weights_only=True,
        verbose=1,
    )

    # Encoder
    bert_individual_size = 128
    bert_individual_encoder = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1", 
        trainable=True,
    )
    
    # Optimizer
    num_train_steps = (epochs*len(tweet_individual_train['input_word_ids']))
    num_warmup_steps = int(0.1*num_train_steps)
    bert_optimizer = optimization.create_optimizer(
        init_lr=learning_rate,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        optimizer_type='adamw'
    )
    
    # Model
    bert_individual_model = test_bert_model(
        bert_individual_encoder, 
        bert_individual_size,
    )
    bert_individual_model.compile(
        optimizer=bert_optimizer, 
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
        metrics=tf.metrics.BinaryAccuracy(),
    )
    
    # Train BERT
    bert_individual_model.fit(
        x=tweet_individual_train,
        y=label_individual_train,
        batch_size=batch_size,
        epochs=epochs,
        callbacks=[bert_individual_checkpoint_callback, tensorboard_callback],
        validation_data=(tweet_individual_val, label_individual_val),
    )

Epoch 1/10
   2/1313 [..............................] - ETA: 10:32 - loss: 0.7859 - binary_accuracy: 0.4688WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0480s vs `on_train_batch_end` time: 0.9142s). Check your callbacks.


1214/1313 [==========================>...] - ETA: 7s - loss: 0.7042 - binary_accuracy: 0.5207

In [ ]:
send_email("Loss testing finished for individual model")

In [17]:
bert_individual_model.evaluate(tweet_individual_val, label_individual_val)

141/141 [==============================] - 6s 46ms/step - loss: 2.3060 - accuracy: 0.5780


[2.3060004711151123, 0.578000009059906]

In [84]:
# Load the above BERT model
# bert_individual_model = bclf.create_bert_model(
#     bert_individual_encoder, 
#     bert_individual_size,
# )
# bert_individual_model.load_weights(
#     f"training/bert_individual/batch32-epoch100/cp.ckpt"
# ).expect_partial()

In [90]:
from joblib import dump, load

# Train Logistic Regression classifier
X_train, y_train = calculate_user_predictions_from_individual_tweets(
    bert_individual_model, 
    tweet_individual_train, 
    label_individual_train,
)
X_train_sorted = np.sort(X_train, axis=1)

clf = LogisticRegression()
clf.fit(X_train_sorted, y_train)
dump(clf, model_path + "logistic_regressor.joblib") 

['training/bert_individual/best-batch_size32-epochs-100-2/logistic_regressor.joblib']

In [92]:
# Evaluate the model
X_val, y_val = calculate_user_predictions_from_individual_tweets(
    bert_individual_model, 
    tweet_individual_val, 
    label_individual_val,
)
X_val_sorted = np.sort(X_val, axis=1)

X_test, y_test = calculate_user_predictions_from_individual_tweets(
    bert_individual_model, 
    tweet_individual_test, 
    label_individual_test,
)
X_test_sorted = np.sort(X_test, axis=1)

pred_val = clf.predict(X_val_sorted)
pred_test = clf.predict(X_test_sorted)

result = f"Validation eval:\n{evaluate_model(pred_val, y_val)}\nTest eval:\n{evaluate_model(pred_test, y_test)}"
send_email(result)
print(result)

Validation eval:
{'true_positives': 19, 'false_positives': 8, 'false_negatives': 5, 'true_negatives': 13, 'accuracy': 0.7111111111111111, 'precision': 0.7037037037037037, 'recall': 0.7916666666666666, 'f1': 0.7450980392156864}
Test eval:
{'true_positives': 14, 'false_positives': 12, 'false_negatives': 4, 'true_negatives': 15, 'accuracy': 0.6444444444444445, 'precision': 0.5384615384615384, 'recall': 0.7777777777777778, 'f1': 0.6363636363636364}


## BERT Tweet Feed Model

In [ ]:
medium_bert_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1"
bert_encoder_feed = hub.KerasLayer(
    medium_bert_url, 
    trainable=True,
)

bert_input_size_feed = 512

In [ ]:
feed_tokenizer = clf.BertTweetFeedTokenizer(bert_encoder_feed, bert_input_size_feed)
bert_model_feed = bclf.create_bert_model(bert_encoder_feed, bert_input_size_feed)
bert_model_feed.compile(Adam(lr=1e-5), 'binary_crossentropy', ['accuracy'])

### Training

In [ ]:
tweet_feed_train = feed_tokenizer.tokenize_input(tweet_train)
label_feed_train = feed_tokenizer.tokenize_labels(label_train)
tweet_feed_val = feed_tokenizer.tokenize_input(tweet_val)
label_feed_val = feed_tokenizer.tokenize_labels(label_val)
tweet_feed_test = feed_tokenizer.tokenize_input(tweet_test)
label_feed_test = feed_tokenizer.tokenize_labels(label_test)

In [ ]:
def bert_loss_testing(model_path, bert_size, bert_encoder_url, X_train, y_train, X_val, y_val):
    setup = [(b, e, lr) 
             for b in [8, 16, 24, 32, 40] 
             for e in [10] 
             for lr in [5e-5, 3e-5, 2e-5, 1e-5]]

    for batch_size, epochs, learning_rate in setup:
        # TensorBoard callback for logging loss
        model_name = f"batch_size{batch_size}-epochs{epochs}-lr{learning_rate}"
        log_dir = model_path + "logs/" + model_name
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

        # Checkpoint to save model
        checkpoint_path = model_path + model_name + "/cp.ckpt"
        checkpoint_callback = ModelCheckpoint(
            filepath=checkpoint_path,
            save_weights_only=True,
            verbose=1,
        )

        # BERT model
        bert_encoder = hub.KerasLayer(
            bert_encoder_url, 
            trainable=True,
        )
        bert_model = bclf.create_bert_model(
            bert_encoder, 
            bert_size,
        )
        bert_model.compile(
            Adam(learning_rate=learning_rate), 
            'binary_crossentropy', 
            ['accuracy'],
        )

        # Train BERT
        bert_model.fit(
            x=X_train,
            y=y_train,
            batch_size=batch_size,
            epochs=epochs,
            callbacks=[checkpoint_callback, tensorboard_callback],
            validation_data=(X_val, y_val),
        )

In [ ]:
bert_loss_testing(
    "training/bert_feed/loss-testing/", 
    bert_input_size_feed,    
    medium_bert_url, 
    tweet_feed_train, 
    label_feed_train, 
    tweet_feed_val, 
    label_feed_val,
)